In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 5GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
train_df=pd.read_csv('/kaggle/input/titanic/train.csv')
train_df.head()

In [ ]:
train_df.info()

In [ ]:
train_df.describe()

In [ ]:
train_df.describe(include=np.object)

In [ ]:
train_df.isnull().sum()

In [ ]:
import matplotlib.pyplot as plt
import seaborn as sns

In [ ]:
sns.distplot(train_df['Age'],bins=15,color='y',rug=True)
plt.show()

In [ ]:
train_df['Embarked'].value_counts()

In [ ]:
sns.countplot(train_df['Embarked'])
plt.show()

In [ ]:
dict_NA={'Age': train_df['Age'].median(),'Embarked': train_df['Embarked'].value_counts().index[0]}
train_df1=train_df.fillna(value=dict_NA)
train_df1.drop('Cabin',axis=1,inplace=True)
train_df1.isnull().sum()

In [ ]:
train_df1['PassengerId'].nunique()

In [ ]:
train_df1['Name'].nunique()

In [ ]:
train_df1['Ticket'].nunique()

In [ ]:
train_df1['Ticket'].value_counts()

In [ ]:
train_df2=train_df1.drop(['PassengerId','Name','Ticket'],axis=1)
train_df2.head()

In [ ]:
train_df2.describe()


In [ ]:
train_df2.describe(include=np.object)

In [ ]:
train_df2['SibSp'].value_counts()

In [ ]:
train_df2['Parch'].value_counts()

In [ ]:
train_df2['Travel_alone']=np.where(train_df2['SibSp']+train_df2['Parch']>0,0,1)
train_df2['Travel_alone'].value_counts()

In [ ]:
train_df2.drop(['SibSp','Parch'],axis=1,inplace=True)
train_df2.head()

In [ ]:
import sys
!{sys.executable} -m pip install pyod

In [ ]:
sns.boxplot(train_df2['Age'],data=train_df2,orient='v')

In [ ]:
sns.boxplot(train_df2['Fare'],data=train_df2,orient='v')

In [ ]:
sns.barplot(x='Sex',y='Survived',data=train_df2)

In [ ]:
print("Percentage of females survived is: {} %." .format((((train_df2['Survived']==1)[train_df2.Sex=='female']).sum()/(train_df2['Survived']==1).sum())*100))

In [ ]:
print("Percentage of male survived is: {} %." .format((((train_df2['Survived']==1)[train_df2.Sex=='male']).sum()/(train_df2['Survived']==1).sum())*100))

In [ ]:
sns.barplot(x='Pclass',y='Survived',data=train_df2)

In [ ]:
print("Percentage of 1st class survived is: {} %." .format((((train_df2['Survived']==1)[train_df2.Pclass==1]).sum()/(train_df2['Survived']==1).sum())*100))

In [ ]:
print("Percentage of 2nd class survived is: {} %." .format(((train_df2['Survived'][train_df2.Pclass==2]).sum()/(train_df2['Survived']==1).sum())*100))

In [ ]:
print("Percentage of 3rd class survived is: {} %." .format(((train_df2['Survived'][train_df2.Pclass==3]).sum()/(train_df2['Survived']==1).sum())*100))

In [ ]:
sns.barplot('Embarked','Survived',data=train_df2)

In [ ]:
print("Percentage of Embarked S survived is: {} %." .format((((train_df2['Survived']==1)[train_df2.Embarked=='S']).sum()/(train_df2['Survived']==1).sum())*100))

In [ ]:
print("Percentage of Embarked C survived is: {} %." .format((((train_df2['Survived']==1)[train_df2.Embarked=='C']).sum()/(train_df2['Survived']==1).sum())*100))

In [ ]:
print("Percentage of Embarked Q survived is: {} %." .format((((train_df2['Survived']==1)[train_df2.Embarked=='Q']).sum()/(train_df2['Survived']==1).sum())*100))

In [ ]:
sns.barplot('Travel_alone','Survived',data=train_df2)

In [ ]:
train_df3=pd.get_dummies(train_df2,columns=['Pclass','Sex','Embarked'],drop_first=True)
train_df3.head()

In [ ]:
features=train_df3.drop('Survived',axis=1)
y=train_df3['Survived']
features.head(3)

In [ ]:
features.describe()

In [ ]:
X=features.values
from pyod.models.knn import KNN
outlier_fraction=0.2
knn_out=KNN(contamination=outlier_fraction)
knn_out.fit(X)
y_out=knn_out.predict(X)
y_out

In [ ]:
n_outliers=np.count_nonzero(y_out==1)
n_outliers

In [ ]:
array=y_out==0
X=X[array,:]
X.shape
#array.shape

In [ ]:
y=y[array]
y.shape


In [ ]:
from sklearn.preprocessing import StandardScaler
scaler=StandardScaler()
scaler.fit(X)
X_scaled=scaler.transform(X)

In [ ]:
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split
from sklearn.metrics import roc_curve
from sklearn.metrics import roc_auc_score
from sklearn.metrics import classification_report
from sklearn.metrics import confusion_matrix

In [ ]:
logreg=LogisticRegression()
X_train,X_test,y_train,y_test=train_test_split(X_scaled,y,test_size=0.3,random_state=28)
logreg.fit(X_train,y_train)
y_pred=logreg.predict(X_test)
logreg.score(X_test,y_test)

In [ ]:
confusion_matrix(y_test,y_pred)

In [ ]:
y_pred_prob=logreg.predict_proba(X_test)[:,1]
fpr,tpr,thresholds=roc_curve(y_test,y_pred_prob)
plt.plot(fpr,tpr)
plt.xlabel("False positive rate")
plt.ylabel("True positive rate")
plt.title("ROC Curve")

In [ ]:
roc_auc_score(y_test,y_pred_prob)

In [ ]:
test_df=pd.read_csv('/kaggle/input/titanic/test.csv')
test_df.head()

In [ ]:
test_df.shape

In [ ]:
test_df.isnull().sum()

In [ ]:
sns.distplot(test_df['Age'],bins=15,color='y',rug=True)
plt.show()

In [ ]:
sns.distplot(test_df['Fare'],bins=15,color='g',rug=True)
plt.show()

In [ ]:
dict_NA1={'Age': test_df['Age'].median(),'Fare': test_df['Fare'].median()}
test_df1=test_df.fillna(value=dict_NA1)
test_df1.drop('Cabin',axis=1,inplace=True)
test_df1.isnull().sum()

In [ ]:
test_df1['PassengerId'].nunique()

In [ ]:
test_df1['Name'].nunique()

In [ ]:
test_df1['Ticket'].nunique()

In [ ]:
test_df2=test_df1.drop(['PassengerId','Name','Ticket'],axis=1)
test_df2.head()

In [ ]:
test_df2['Travel_alone']=np.where(test_df2['SibSp']+test_df2['Parch']>0,0,1)
test_df2['Travel_alone'].value_counts()

In [ ]:
test_df2.drop(['SibSp','Parch'],axis=1,inplace=True)
test_df2.head()

In [ ]:
test_df3=pd.get_dummies(test_df2,columns=['Pclass','Sex','Embarked'],drop_first=True)
test_df3.head()

In [ ]:
test_features=test_df3.values

In [ ]:
scaler_test=StandardScaler()
scaler_test.fit(test_features)
X_test_scaled=scaler_test.transform(test_features)
y_test_pred=logreg.predict(X_test_scaled)


In [ ]:
y_test_pred

In [ ]:
y_test_pred.shape

In [ ]:
test_df1['PassengerId']

In [ ]:
submission_df=pd.DataFrame({'PassengerId':test_df1['PassengerId'],'Survived':y_test_pred})

In [ ]:
submission_df.to_csv('submission.csv',index=False)